In [1]:

## classic pydata stack
import os 
import numpy as np
from tqdm import tqdm
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import statsmodels.api as sm
import statsmodels.formula.api as smf

%matplotlib inline 

plt.style.use('ggplot')
plt.rcParams['figure.figsize'] = (15,7)



## torch
import torch
from torch.utils.data import Dataset
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

## SEEDING

torch.manual_seed(1)


REBUILD_DATA = True

## Report notes

- need to filter out signals that are too long because the length distribution between the two datasets is very important
- need to balance datasets because one dataset is much bigger than the other
- process the data such that an RNN or any NN can process it
- 

In [2]:
class Input():
    def __init__(self, raw_series,num_blocks,label):
        """ Initilaizes an input object from a raw time series i.e. an input suitable to feed to a recurrent neural network

        Args:
            raw_series (numpy array of shape (num_timesteps,2)): raw time series from npy data i.e. arr[0] where arr = np.load("data.npy")
            num_blocks ([type]): number of "feature blocks" into which the time series will be sliced i.e the number of of times we need to feed 
            to the LSTM to train on the entire time series
            label ([type]): Whether it was a "00" backbone (label:0) or a "66" backbone (label:1)
        """

        self.label = label
        self.input = self.process(raw_series,num_blocks)


    def process(self,raw_series,num_blocks):
        """ Function that does the entire processing of going from raw time series to a suitable input to feed to a recurrent neural network

        Args:
            raw_series (numpy array of shape (num_timesteps,2)): raw time series from npy data i.e. arr[0] where arr = np.load("data.npy")
            num_blocks ([type]): number of "feature blocks" into which the time series will be sliced i.e the number of of times we need to feed 
            to the LSTM to train on the entire time series

        Returns:
            np.ndarray: array of features from a single raw time series instance
        """


        # stores the processed time series
        res = np.array([])

        ## returns a list of transformed time series (current list: normal. lowpass filtered, highpass filtered)
        instances = self.transform(raw_series)


        for instance in instances:
            ## chunks an instance of a time series into blocks and extract feature from each block
            extracted = self.extract_features(instance,num_blocks)
            res = np.concatenate((res,extracted),axis=None)

        return res


    def transform(self,raw_series):
        """ Given a raw time series, outputs several transformations applied to it
            Transformations may be filtering, projecting, ...

        Args:
            raw_series numpy.ndarray : 1 dimensional array representing the current values

        Returns:
            List(numpy.ndarray): list of all transformations
        """

        res = [raw_series]

        return res

    def extract_features(self,instance, num_blocks):

        """ From a time series, divides it into num_blocks blocks and from each block, extract numerical features usable for a neural network

        Args:
            instance (numpy.ndarray): 1D array containing numerical values
            num_blocks (int): number of "feature blocks" into which the time series will be sliced i.e the number of of times we need to feed 
            to the LSTM to train on the entire time series


        Returns:
            numpy.ndarray: 1D array of length num_blocks*num_features_per_block containing all the features from a time series
        """

        res = np.array([])
        length = len(instance)
        # divide the length by num_blocks to get block_size
        block_size, remainder  = divmod(length,num_blocks)


        # iterating over each block and extracting features
        for i in range(num_blocks):

            curr = instance[block_size*i: block_size*(i+1)]
            # get features from block (mean, std, length, ...)
            features = self.features(curr)
            res = np.concatenate((res, features),axis=None)


        ## get the remainder of the time series
        ##curr = instance[block_size*num_blocks:]
        ##features = self.features(curr)
        ##res = np.concatenate((res,features),axis=None)

        return res

    def features(self,instance):
        """
        From a block of a time series, extracts numerical features usable for a neural network
        Args:
            instance (numpy.ndarray): 1D array containing numerical values 
        """
        res = np.array([])

        # list of functions applied to the array for feature extraction
        functions = [np.mean,np.median,np.std,np.min,np.max,len]

        for func in functions:
            res = np.concatenate((res,func(instance)),axis=None)

        

        return res        

In [3]:
from torch.utils.data import Dataset

class PolymerDataset(Dataset):

    ## These functions are necessary to define an iterator usable by Pytorch

    def __init__(self, data_paths,num_blocks, lstm=False, seed=10):
        super().__init__()
        self.process(data_paths,num_blocks,lstm,seed)

    def __len__(self):
        return len(self.labels)
    
    def __getitem__(self, idx):
        return self.data[idx], self.labels[idx]


    def process(self,data_paths, num_blocks,lstm,seed):
        """ Processes the two datasets in the aim of not having bias catchable by the neural network:
        - filtering signals that are too long and too short
        - balancing the two datasets, resulting in the two classes each representing 50% of the data
        - Process each of the raw time series current into suitable inputs
        - Output a dataset where each row represents a suitable input for a NN derived from the raw time series

        Args:
            data_paths (list[string]): Should be a list of length 2 containing the paths of the data to be loaded 
            num_blocks (int): number of "feature blocks" into which a raw time series will be sliced i.e the number of of times we need to feed the
            to the LSTM to train on the entire time series
            seed (int): for setting the seed

        """
        
        raw_data = [np.load(data_path, allow_pickle=True) for data_path in data_paths]
        labels = [0,1]

    ## balance the dataset by removing signals that are too short or too long
    ## first we build the dataframe to know the lengths of the time series

        len_series = []

        for data in raw_data:
            lengths= []
            for row in data:
                ## length of time series
                lengths.append(row.shape[0])

            len_series.append(pd.Series(lengths))

        ## enforces that the first dataset is the smaller one in total size
        ## such that we can apply our balancing operations generally
        if len(len_series[0]) > len(len_series[1]):
            len_series.reverse()
            raw_data.reverse()
            labels.reverse()

        ## filter the dataset and remove signals that are:
        ## too short i.e. < len_series[0].quantile(0.1)
        ## too long i.e. > len_series[0].quantile(0.9)
        for i in range(2):
            mask = (len_series[i] > len_series[0].quantile(0.1)) & (len_series[i] < len_series[0].quantile(0.9))
            raw_data[i] = raw_data[i][mask]

        ## most likely, one dataset is still bigger than the other one
        ## therefore, we randomly sample data from the bigger dataset to create a new dataset of the same size as the small one 
        np.random.seed(seed=seed)

        # making sure the smallest dataset is the first one
        if len(raw_data[0]) > len(raw_data[1]):
            raw_data.reverse()
            labels.reverse()

        # randomly sampling and making a balanced dataset
        raw_data[1]  = np.random.permutation(raw_data[1])[:len(raw_data[0])]
        data=[]
        data_labels=[]
        
        ## using our Input class to build the entire dataset and extracting features from each row
        for index, raw_data in enumerate(raw_data):
            for raw_series in raw_data:
                processed_series = Input(raw_series=raw_series,num_blocks=num_blocks,label=labels[index])
                data.append(processed_series.input)
                data_labels.append(labels[index])
        data = np.array(data)

        #normalizing features
        data = (data - data.mean(axis=0)) / data.std(axis=0)

        data = torch.Tensor(data).float()

        ## if lstm is true, set up the data such that it can easily be fed into a lstm
        if lstm:
            data = data.view((data.shape[0],num_blocks,-1))

        self.data = data
        self.labels = torch.Tensor(np.array(data_labels)).long()

        return self


    

In [4]:
from torch.utils.data import DataLoader

class LSTM(nn.Module):

    def __init__(self, input_dim, num_layers, hidden_dim):
        """ creates a lstm neural network

        Args:
            input_dim (int)): Defines the dimension of the input x, should be equal to the number of features extracted per block
            num_layers(int): Defines the number of LSTM layers, should be equal to num_blocks
            hidden_dim (int): defines the number of features in the hidden states
        """
        super(LSTM, self).__init__()
        self.lstm = nn.LSTM(input_size= input_dim, num_layers=num_layers, hidden_size=hidden_dim,batch_first=True)
        self.fc1 =nn.Linear(hidden_dim, 2)

    
    def forward(self, input):
        """ Forward pass of our network

        Args:
            input ([type]): should be our current time series preprocessed with shape(num_blocks, num_features) 
            where num_blocks is the number of blocks in which we have divided our time series and  num_features is the number of feature per block
        """
        num_blocks=input.shape[0]
        
        ## the LSTM output are the hidden states values for all hidden states while processing the sequence
        lstm_out, _ = self.lstm(input)

        ## we only want last hidden states values
        lstm_out = lstm_out[:,-1,:]
        ## passing through MLP and softmax
        last = self.fc1(lstm_out.view(num_blocks,-1))
        scores = F.log_softmax(last,dim=1)

        return scores

    def predict(self, test_data):
        probs = self.forward(X)
        preds = torch.argmax(probs, dim=1, keepdim=False)
        return preds

    def train(dataset, num_features, num_blocks, hidden_dim, num_epochs, batch_size):

        data_loader = DataLoader(dataset, batch_size=batch_size, shuffle=True)
        model = LSTM(input_dim = num_features, num_layers= num_blocks ,hidden_dim = hidden_dim)
        loss_function = torch.nn.NLLLoss()
        optimizer = torch.optim.SGD(model.parameters(), lr=0.1)

        for epoch in range(num_epochs):
            num_correct = 0
            for X, y in iter(data_loader):
                model.zero_grad()
                probs = model(X)
                loss = loss_function(probs, y)
                loss.backward()
                optimizer.step()
                preds = torch.argmax(probs, dim=1, keepdim=False)
                num_correct += (preds == y).sum()
            print(f'epoch={epoch}/{num_epochs - 1}, loss={loss}, accuracy={num_correct*100/len(dataset)}')

        return model

In [5]:
num_blocks=3
dataset = PolymerDataset(data_paths=["AA00400AA.npy","AA66466AA.npy"],num_blocks=num_blocks,lstm=True)
num_features = dataset.data[0].shape[1]

from torch.utils.data import random_split

train_size = int(0.8 * len(dataset))
test_size = len(dataset) - train_size
train_data, test_data = random_split(dataset, [train_size, test_size])


In [6]:

model = LSTM.train(dataset=train_data, num_features=num_features, num_blocks=num_blocks, hidden_dim=3, num_epochs=20, batch_size=64)

epoch=0/19, loss=0.7252147793769836, accuracy=50.019859313964844
epoch=1/19, loss=0.7017585635185242, accuracy=49.46820068359375
epoch=2/19, loss=0.693034827709198, accuracy=49.99779510498047
epoch=3/19, loss=0.6864487528800964, accuracy=50.98636245727539
epoch=4/19, loss=0.6845557689666748, accuracy=50.86720657348633
epoch=5/19, loss=0.6677550673484802, accuracy=51.89549255371094
epoch=6/19, loss=0.6874570846557617, accuracy=52.65457534790039
epoch=7/19, loss=0.7309290766716003, accuracy=55.730613708496094
epoch=8/19, loss=0.6609702706336975, accuracy=65.7884292602539
epoch=9/19, loss=0.40221869945526123, accuracy=83.6356430053711
epoch=10/19, loss=0.2085624486207962, accuracy=89.24488830566406
epoch=11/19, loss=0.029638992622494698, accuracy=89.50086212158203
epoch=12/19, loss=0.6779641509056091, accuracy=89.5802993774414
epoch=13/19, loss=0.08710702508687973, accuracy=89.81861877441406
epoch=14/19, loss=0.6329165101051331, accuracy=90.33055114746094
epoch=15/19, loss=0.1831070929765

In [17]:
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score,confusion_matrix

data_loader = DataLoader(test_data, batch_size=64, shuffle=False)

predictions = np.array([])
labels = np.array([])

with torch.no_grad():
    for X, y in iter(data_loader):
        probs = model(X)
        preds = torch.argmax(probs, dim=1, keepdim=False)
        predictions = np.concatenate((predictions,preds), axis=None)
        labels= np.concatenate((labels,y),axis=None)


accuracy = accuracy_score(labels,predictions)
f1 = f1_score(labels,predictions)
precision = precision_score(labels,predictions)
recall = recall_score(labels,predictions)

names =["Accuracy", "F1 Score", "Precision", "Recall"]
functions = [accuracy_score, f1_score, precision_score, recall_score]

for name, func in zip(names,functions):
    score = func(labels,predictions)
    print(f"{name}: {score*100:.2f}%")




Accuracy: 89.78%
F1 Score: 89.47%
Precision: 91.82%
Recall: 87.23%
